In [10]:
import pandas as pd
import os as os
from binarytree import Node

df = pd.read_csv('mutations.csv')
dfA = pd.read_csv('mutations.csv') #dataframe duplicate for mutation A
dfB = pd.read_csv('mutations.csv') #dataframe duplicate for mutation B
accDf = df
randomDF = pd.read_csv('mutations.csv')

# calculates the accuracy value for a mutation 
def mutAcc(rowNum, colnum, tempDF):
    acc = 0; 
    if(colnum > 0 and colnum < 1411):
        acc = ((true_positives(rowNum, colnum, tempDF) + true_negatives(rowNum, colnum, tempDF))
               / (true_positives(rowNum, colnum, tempDF) + true_negatives(rowNum, colnum, tempDF) +
                  false_negatives(rowNum, colnum, tempDF) + false_positives(rowNum, colnum, tempDF)))
    return acc;


# places the accuracy values for each mutation in a temporary data frame 
# row num is the row we add accuracy value too
# temp DF is the dataframe we want to add a accuracy value to
def addAccDF(rowNum, tempDF):
    for i in range (1411):
        if(i == 0):
            continue
        else:
            tempDF.iloc[rowNum, i] = mutAcc(rowNum, i, tempDF)

# row Num is the total rows in the dataframe
def true_positives(rowNum, colnum, tempDF):
    tp = 0
    if(colnum > 0 and colnum < 1411):
       for i in range (rowNum): 
         if((tempDF.iloc[i,0][0] == 'C') and (tempDF.iloc[i, colnum] == 1.0)):
               tp+=1
    return tp

# gets the number of true positives when using the classifier
def classifier_TP(rowNum, tempDF, tempbt):
    tp = 0
    for i in range (rowNum): 
        if((tempDF.iloc[i,0][0] == 'C') and (num_classifier(tempDF.iloc[i,0], tempbt) == 1.0)):
            tp+=1
    return tp

def phiclassifier_TP(temp_df, temp_bt):
    tp = 0 
    for i in range (len(temp_df.index)):
        if((temp_df.iloc[i,0][0] == 'C') and (phi_classifier(temp_df.iloc[i,0], temp_df, temp_bt) == 1.0)):
            tp+=1
    return tp

def false_positives(rowNum, colnum, tempDF):
    fp = 0
    if(colnum > 0 and colnum < 1411):
       for i in range (rowNum): 
         if((tempDF.iloc[i,0][0] == 'N') and (tempDF.iloc[i, colnum] == 1.0)):
               fp+=1
    return fp

def classifier_FP(rowNum, tempDF, tempbt):
    fp = 0
    for i in range (rowNum): 
        if((tempDF.iloc[i,0][0] == 'N') and (num_classifier(tempDF.iloc[i,0], tempbt) == 1.0)):
            fp+=1
    return fp

# Gets total false positives when using a model produced by phi
def phiclassifier_FP(temp_df, temp_bt):
    fp = 0 
    for i in range (len(temp_df.index)):
        if((temp_df.iloc[i,0][0] == 'N') and (phi_classifier(temp_df.iloc[i,0], temp_df, temp_bt) == 1.0)):
            fp+=1
    return fp

def true_negatives(rowNum, colnum, tempDF):
    tn = 0
    if(colnum > 0 and colnum < 1411):
       for i in range (rowNum): 
         if((tempDF.iloc[i,0][0] == 'N') and (tempDF.iloc[i, colnum] == 0.0)):
            tn+=1
    return tn

def classifier_TN(rowNum, tempDF, tempbt):
    tn = 0
    for i in range (rowNum): 
        if((tempDF.iloc[i,0][0] == 'N') and (num_classifier(tempDF.iloc[i,0], tempbt) == 0.0)):
            tn+=1
    return tn

# Gets total true positives when using a model produced by phi
def phiclassifier_TN(temp_df, temp_bt):
    tn = 0 
    for i in range (len(temp_df.index)):
        if((temp_df.iloc[i,0][0] == 'N') and (phi_classifier(temp_df.iloc[i,0], temp_df, temp_bt) == 0.0)):
            tn+=1
    return tn

def false_negatives(rowNum, colnum, tempDF):
    fn = 0
    if(colnum > 0 and colnum < 1411):
       for i in range (rowNum): 
         if((tempDF.iloc[i,0][0] == 'C') and (tempDF.iloc[i, colnum] == 0.0)):
            fn+=1
    return fn

def classifier_FN(rowNum, tempDF, tempbt):
    fn = 0
    for i in range (rowNum): 
        if((tempDF.iloc[i,0][0] == 'C') and (num_classifier(tempDF.iloc[i,0], tempbt) == 0.0)):
            fn+=1
    return fn

# Gets total true positives when using a model produced by phi
def phiclassifier_FN(temp_df, temp_bt):
    fn = 0 
    for i in range (len(temp_df.index)):
        if((temp_df.iloc[i,0][0] == 'C') and (phi_classifier(temp_df.iloc[i,0], temp_df, temp_bt) == 0.0)):
            fn+=1
    return fn

#helper function to get correct 
def TPtakeFP(rowNum, tempDF):
    for i in range (1411):
        if(i == 0):
            continue
        else:
            tempDF.iloc[rowNum, i] = true_positives(rowNum, i, tempDF) - false_positives(rowNum, i, tempDF)
            
# This code will add all TP - FP values to a dataframe 
def add_TPFP_df(row_total, tempDF):
    tempDF = tempDF.append({tempDF.columns[0]:'TP-FP'}, ignore_index = True)
    TPtakeFP(row_total ,tempDF)
    
    return tempDF

def is_truePos(rowNum, colNumber, tempDF):
    if((tempDF.iloc[rowNum,0][0] == 'C') and (tempDF.iloc[rowNum, colNumber] == 1.0)):
        return True 
    else:
        return False

def is_trueNeg(rowNum, colNumber, tempDF):
    if((tempDF.iloc[rowNum,0][0] == 'N') and (tempDF.iloc[rowNum, colNumber] == 0.0)):
        return True 
    else:
        return False

def is_falseNeg(rowNum, colNumber, tempDF):
    if((tempDF.iloc[rowNum,0][0] == 'C') and (tempDF.iloc[rowNum, colNumber] == 0.0)):
        return True 
    else:
        return False

def is_falsePos(rowNum, colNumber, tempDF):
    if((tempDF.iloc[rowNum,0][0] == 'N') and (tempDF.iloc[rowNum, colNumber] == 1.0)):
        return True 
    else:
        return False

# THIS FUNCTION DOES NOT WORK CORRECTLY! Not using right now
def group_node(tempDF, mutCol , tempDF2):
    for i in range(162):
        if is_truePos(i , mutCol, tempDF2):
            tempDF.append({'Group A':tempDF2.iloc[i, 0]}, ignore_index = True)
        elif is_trueNeg(i, mutCol, tempDF2):
            tempDF.append({'Group B':tempDF2.iloc[i, 0]}, ignore_index = True)
        elif is_falseNeg(i, mutCol, tempDF2):
            tempDF.append({'Group B':tempDF2.iloc[i, 0]}, ignore_index = True)
        else:
            tempDF.append({'Group A':tempDF2.iloc[i, 0]}, ignore_index = True)

def search_mutIndex(target, tempDF):
    for i in range(1411):
        if (tempDF.columns[i] == target):
            return i;
        
    return -1   

def classifier(tempInput):
    found = False
    
    for i in range(162):
        if (tempInput == df.iloc[i,0]):
            found = True
            print ("success")
            
            
            #classification
            if(df.iloc[i, 107] == 1.0):
                if(df.iloc[i, 314] == 1.0):
                    return print ("The patient in A: ", tempInput, "is classifed as Cancer")   
                else:
                    return print ("The patient in A: ", tempInput, "is classifed as Non Cancer")
            else: 
                if (df.iloc[i, 173] == 1.0):
                     return print ("The patient in B: ", tempInput, "is classifed as Cancer")
                else:
                     return print ("The patient in B: ", tempInput, "is classifed as Non Cancer")
    
    
    if (found == False):
        print("This is not a valid patient")
        return


def num_classifier(tempInput, temp_bt):
    #find the mutations that partition
    root_index = temp_bt.value
    left_index = temp_bt.left.value
    right_index = temp_bt.right.value 
    
    found = False
    # find the sample and when found classify it
    for i in range(162):
        if (tempInput == df.iloc[i,0]):
            found = True
            #classification
            if(df.iloc[i, root_index] == 1.0):
                if(df.iloc[i, left_index] == 1.0):
                    return 1.0   
                else:
                    return 0.0
            else: 
                if (df.iloc[i, right_index] == 1.0):
                     return 1.0
                else:
                     return 0.0
    
    
    if (found == False):
        return


def phi_splitDF(tempdf, mut_Index):
    root_L = tempdf
    root_R = tempdf
        
    split_df = pd.DataFrame(columns = ['Root Left', 'Root Right'])
    
    for i in range(len(tempdf.index)):
        if(tempdf.iloc[i, mut_Index] == 1.0):
            split_df = split_df.append({'Root Left':tempdf.iloc[i,0]}, ignore_index = True)
        elif(tempdf.iloc[i, mut_Index] == 0.0):
            split_df = split_df.append({'Root Right':tempdf.iloc[i,0]}, ignore_index = True)
        else:
            continue
                
    root_L = root_L.rename(columns={root_L.columns[0]: "Root Left"})
    root_R = root_R.rename(columns={root_R.columns[0]: "Root Right"})
    root_L = pd.merge(root_L, split_df, on="Root Left")
    root_R = pd.merge(root_R, split_df, on="Root Right")
    root_L = root_L.drop(columns=["Root Right"])
    root_R = root_R.drop(columns=["Root Left"]) 
        
    tlist = [root_L, root_R]
        
    return tlist


def phi_classifier(tempInput, rootdf, temp_bt):
    #find the mutations that partition
    root_index = temp_bt.value
    left_index = temp_bt.left.value
    right_index = temp_bt.right.value
    tlist_Root = phi_splitDF(rootdf, root_index)
    root_L = tlist_Root[0]
    root_R = tlist_Root[1]
    
    
    tlist_A = phi_splitDF(root_L, left_index)
    tlist_B = phi_splitDF(root_R, right_index)
    
    found = False
    # find the sample and when found classify it
    
    for i in range(162):
        if (tempInput == df.iloc[i,0]):
            found = True
            #classification
            
            if(df.iloc[i, root_index] == 1.0):
                if(df.iloc[i, left_index] == 1.0):
                    if((act_totalC(tlist_A[0])) > (act_totalNC(tlist_A[0]))):
                            return 1.0
                    else: 
                            return 0.0 
                else:
                    if(((act_totalC(tlist_A[1])) > (act_totalNC(tlist_A[1])))):
                        return 1.0
                    else:
                        return 0.0   
            else:
                if(df.iloc[i, right_index] == 1.0):
                    if((act_totalC(tlist_B[0])) > (act_totalNC(tlist_B[0]))):
                        return 1.0
                    else: 
                        return 0.0 
                elif(((act_totalC(tlist_B[1])) > (act_totalNC(tlist_B[1])))): 
                    return 1.0
                else:
                    return 0.0
                
    
    if (found == False):
        return

def eval(row_num, tempdf, temp_bt, classifierFunc):
    temp_df = pd.DataFrame(index = ['Classifier'], columns = ['TP', 'TN', 'FP', 'FN', 'Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'Miss Rate', 'False Discovery Rate','False Omission Rate'])
    
    if (classifierFunc == num_classifier):
        ctp = classifier_TP(row_num, tempdf, temp_bt)
        ctn = classifier_TN(row_num, tempdf, temp_bt)
        cfp = classifier_FP(row_num, tempdf, temp_bt)
        cfn = classifier_FN(row_num, tempdf, temp_bt)
    if(classifierFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt)
        cfn = phiclassifier_FN(tempdf, temp_bt)
        
    pop = ctp + ctn + cfp + cfn
    temp_df.iloc[0, 0] = ctp
    temp_df.iloc[0, 1] = ctn
    temp_df.iloc[0, 2] = cfp
    temp_df.iloc[0, 3] = cfn
    if((ctp + ctn) != 0):
        temp_df.iloc[0, 4] = (ctp + ctn)/(pop)
    if((ctp + cfn) != 0):
        temp_df.iloc[0,5] =  (ctp)/(ctp + cfn)
    if((ctn + cfp) != 0):
        temp_df.iloc[0,6] = (ctn)/(ctn + cfp)
    if((ctp + cfp) != 0):
        temp_df.iloc[0, 7] = (ctp) / (ctp + cfp)
    if((cfn + ctp) != 0):
        temp_df.iloc[0, 8] = (cfn) / (cfn + ctp)
    if((cfp + ctp) != 0):
        temp_df.iloc[0, 9] = (cfp)/(cfp + ctp)
    if((cfn + ctn) != 0):
        temp_df.iloc[0, 10] = (cfn)/(cfn + ctn)
    return temp_df

def phi_mutData(mutName, root_df):
    root_L = root_df
    root_R = root_df
    split_df = pd.DataFrame(columns = ['Root Left', 'Root Right'])
    temp_df = pd.DataFrame(columns = ['Mutation','n-tL', 'n-tR', 'n-tL-C', 'n-tL-NC','n-tR-C','n-tR-NC', 'p-L', 'p-R', 'p-C-tl', 'p-NC-tl', 'p-C-tr', 'p-NC-tr','CompOne','Qcomp','Phi'])
    mut_Index = search_mutIndex(mutName, root_df)
        
    #getting right and left groups
    if(mut_Index > 0):
        for i in range(len(root_df.index)):
            if(root_df.iloc[i, mut_Index] == 1.0):
                split_df = split_df.append({'Root Left':root_df.iloc[i,0]}, ignore_index = True)
            elif(root_df.iloc[i, mut_Index] == 0.0):
                split_df = split_df.append({'Root Right':root_df.iloc[i,0]}, ignore_index = True)
            else:
                continue
        root_L = root_L.rename(columns={root_L.columns[0]: "Root Left"})
        root_R = root_R.rename(columns={root_R.columns[0]: "Root Right"})
        root_L = pd.merge(root_L, split_df, on="Root Left")
        root_R = pd.merge(root_R, split_df, on="Root Right")
        root_L = root_L.drop(columns=["Root Right"])
        root_R = root_R.drop(columns=["Root Left"])  
    
    temp_df = temp_df.append({'Mutation': mutName}, ignore_index=True)
    temp_df.iloc[0,1] = num_samples(root_L)
    temp_df.iloc[0,2] = num_samples(root_R)
    temp_df.iloc[0,3] = act_totalC(root_L)
    temp_df.iloc[0,4] = act_totalNC(root_L)
    temp_df.iloc[0,5] = act_totalC(root_R)
    temp_df.iloc[0,6] = act_totalNC(root_R)
    temp_df.iloc[0,7] = prob_Sample(root_L, root_df)
    temp_df.iloc[0,8] = prob_Sample(root_R, root_df)
    temp_df.iloc[0,9] = prob_selectC(root_L)
    temp_df.iloc[0,10] = prob_selectNC(root_L)
    temp_df.iloc[0,11] = prob_selectC(root_R)
    temp_df.iloc[0,12] = prob_selectNC(root_R)
    temp_df.iloc[0,13] = compOne_phi(root_L, root_R, root_df)
    temp_df.iloc[0,14] = Qcomp_phi(root_L, root_R)
    temp_df.iloc[0,15] = phi_functionMut(root_L, root_R, root_df)
            
                           
    return temp_df

def tt_mutData_Phi(tt_ser, rootdf):
    temp_df = pd.DataFrame(columns = ['Mutation','n-tL', 'n-tR', 'n-tL-C', 'n-tL-NC','n-tR-C','n-tR-NC', 'p-L', 'p-R', 'p-C-tl', 'p-NC-tl', 'p-C-tr', 'p-NC-tr','CompOne','Qcomp','Phi'])
    for i in range (10):
        plug_df = phi_mutData(tt_ser.index[i], rootdf)
        temp_df = temp_df.append(plug_df,ignore_index=True)
    return temp_df

# function to get the average evaluation, this could have bugs!!!!! WARNING
def ave_eval(row_total, tempdf_one, tempdf_two, tempdf_three, temp_bt1, temp_bt2, temp_bt3, cFunc):
    av_df = pd.DataFrame(index = ['Classifier'], columns = ['TP', 'TN', 'FP', 'FN', 'Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'Miss Rate', 'False Discovery Rate','False Omission Rate'])
    div = 3
    
    if (cFunc == num_classifier):
        ctp = ((classifier_TP(row_total, tempdf_one , temp_bt1) + classifier_TP(row_total, tempdf_two , temp_bt2) + classifier_TP(row_total, tempdf_three , temp_bt3))/div)
        ctn = ((classifier_TN(row_total, tempdf_one , temp_bt1) + classifier_TN(row_total, tempdf_two , temp_bt2) + classifier_TN(row_total,tempdf_three , temp_bt3))/div)
        cfp = ((classifier_FP(row_total, tempdf_one , temp_bt1) + classifier_FP(row_total, tempdf_two , temp_bt2) + classifier_FP(row_total,tempdf_three , temp_bt3))/div)
        cfn = ((classifier_FN(row_total, tempdf_one , temp_bt1) + classifier_FN(row_total, tempdf_two , temp_bt2) + classifier_FN(row_total,tempdf_three , temp_bt3))/div)
    elif(cFunc == phi_classifier):
        ctp = ((phiclassifier_TP(tempdf_one, temp_bt1) + phiclassifier_TP(tempdf_two, temp_bt2) + phiclassifier_TP(tempdf_three, temp_bt3))/div)
        ctn = ((phiclassifier_TN(tempdf_one ,temp_bt1) + phiclassifier_TN(tempdf_two, temp_bt2) + phiclassifier_TN(tempdf_three, temp_bt3))/div)
        cfp = ((phiclassifier_FP(tempdf_one, temp_bt1) + phiclassifier_FP(tempdf_two, temp_bt2) + phiclassifier_FP(tempdf_three, temp_bt3))/div)
        cfn = ((phiclassifier_FN(tempdf_one, temp_bt1) + phiclassifier_FN(tempdf_two, temp_bt2) + phiclassifier_FN(tempdf_three, temp_bt3))/div)
    else:
        return print("Ave_eval could not find classifier")
        
    av_df.iloc[0, 0] = ctp
    av_df.iloc[0, 1] = ctn
    av_df.iloc[0, 2] = cfp
    av_df.iloc[0, 3] = cfn
    av_df.iloc[0, 4] = ((accuracy(row_total, tempdf_one , temp_bt1, cFunc) + accuracy(row_total, tempdf_two , temp_bt2, cFunc) + accuracy(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0,5] =  ((sensitivity(row_total, tempdf_one , temp_bt1, cFunc) + sensitivity(row_total, tempdf_two , temp_bt2, cFunc) + sensitivity(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0,6] =  ((specificity(row_total, tempdf_one , temp_bt1, cFunc) + specificity(row_total, tempdf_two , temp_bt2, cFunc) + specificity(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0, 7] = ((precision(row_total, tempdf_one , temp_bt1, cFunc) + precision(row_total, tempdf_two , temp_bt2, cFunc) + precision(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0, 8] = ((miss_rate(row_total, tempdf_one , temp_bt1, cFunc) + miss_rate(row_total, tempdf_two , temp_bt2, cFunc) + miss_rate(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0, 9] = ((falseDR(row_total, tempdf_one , temp_bt1, cFunc) + falseDR(row_total, tempdf_two , temp_bt2, cFunc) + falseDR(row_total,tempdf_three , temp_bt3, cFunc))/div)
    av_df.iloc[0, 10] =((falseOR(row_total, tempdf_one , temp_bt1, cFunc) + falseOR(row_total, tempdf_two , temp_bt2, cFunc) + falseOR(row_total,tempdf_three , temp_bt3, cFunc))/div)
    return av_df

def accuracy(row_num, tempdf, temp_bt, cFunc):
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("accuracy could not find classifier")
        
    pop = ctp + ctn + cfp + cfn
    acc = (ctp + ctn)/(pop)
    return acc

def sensitivity(row_num, tempdf, temp_bt, cFunc):
    sens = -1
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("Sensitivity could not find classifier")
    
    if((ctp + cfn) != 0):
        sens = (ctp)/(ctp + cfn)  
    return sens

def specificity(row_num, tempdf, temp_bt, cFunc):
    spec = -1
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("could not find classifier")
    
    if((ctn + cfp) != 0):
        spec = (ctn)/(ctn + cfp)
    return spec

def precision(row_num, tempdf, temp_bt, cFunc):
    prec = -1
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("could not find classifier")
    
    if((ctp + cfp) != 0):
        prec = (ctp) / (ctp + cfp)
    return prec


def miss_rate(row_num, tempdf, temp_bt, cFunc):
    mr = -1
    
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("could not find classifier")
    
    if((cfn + ctp) != 0):
        mr = (cfn) / (cfn + ctp)
    return mr

def falseDR(row_num, tempdf, temp_bt, cFunc):
    dr = -1 
    
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("could not find classifier")
    
    if((cfp + ctp) != 0):
        dr = (cfp)/(cfp + ctp)
    return dr

def falseOR(row_num, tempdf, temp_bt, cFunc):
    fOR = -1
    
    if(cFunc == num_classifier):
        ctp = classifier_TP(row_total, tempdf, temp_bt)        
        ctn = classifier_TN(row_total, tempdf, temp_bt)
        cfp = classifier_FP(row_total, tempdf, temp_bt)
        cfn = classifier_FN(row_total, tempdf, temp_bt)
    elif(cFunc == phi_classifier):
        ctp = phiclassifier_TP(tempdf, temp_bt)
        ctn = phiclassifier_TN(tempdf, temp_bt)
        cfp = phiclassifier_FP(tempdf, temp_bt) 
        cfn = phiclassifier_FN(tempdf, temp_bt)
    else:
        return print("could not find classifier")
    
    if((cfn + ctn) != 0):
        fOR = (cfn)/(cfn + ctn)
    return fOR

def make_tree(row_total, tempdf):
    # Create duplicate dataframes for group A and B, keep track of their number of rows 
    df_splitA = tempdf
    df_splitB = tempdf
    split_rtotal = row_total
    
    # Check if the temp df has TP-FP row, if it does drop the row in the group DFs and update the  split total 
    if(tempdf.iloc[((len(tempdf.index)) - 1), 0] == 'TP-FP'):
        df_splitA = df_splitA.drop(((len(tempdf.index)) - 1))
        df_splitB = df_splitB.drop(((len(tempdf.index)) - 1))
        split_rtotal -= 1 #54 delete later
    else: # This else statement has not been properly tested might be buggy!!!! Warning
        tempdf = add_TPFP_df(row_total, tempdf)
        row_total += 1
        
            
    # Get the top TP-FP values in the tempdf, we will need this to get they key mutations, create dataframe for split data df_split
    index_last = row_total - 1
    df_split = pd.DataFrame(columns = ['Group A', 'Group B'])
    tempSer = tempdf.iloc[index_last, 1:1411].sort_values(ascending = False)
    data = [[tempSer.index[0], tempSer.iloc[0]], [tempSer.index[1], tempSer.iloc[1]], [tempSer.index[2], tempSer.iloc[2]], [tempSer.index[3], tempSer.iloc[3]], [tempSer.index[4], tempSer.iloc[4]], [tempSer.index[5], tempSer.iloc[5]],
        [tempSer.index[6], tempSer.iloc[6]], [tempSer.index[7], tempSer.iloc[7]], [tempSer.index[8], tempSer.iloc[8]], [tempSer.index[9], tempSer.iloc[9]]]

    root_name = tempSer.index[0] 
    
    # Partition the samples intto groups A and B 
    for i in range(index_last):
        if is_truePos(i ,search_mutIndex(root_name, tempdf), tempdf):
            df_split = df_split.append({'Group A':tempdf.iloc[i, 0]}, ignore_index = True)
        elif is_trueNeg(i, search_mutIndex(root_name, tempdf), tempdf):
            df_split = df_split.append({'Group B':tempdf.iloc[i, 0]}, ignore_index = True)
        elif is_falseNeg(i, search_mutIndex(root_name, tempdf), tempdf):
             df_split = df_split.append({'Group B':tempdf.iloc[i, 0]}, ignore_index = True)
        elif is_falsePos(i, search_mutIndex(root_name, tempdf), tempdf):
            df_split = df_split.append({'Group A':tempdf.iloc[i, 0]}, ignore_index = True)
        else:
            continue 
        
    # Setting up the dataframes for groups A and B, renaming, merging and then dropping extra stuff     
    df_splitA = df_splitA.rename(columns={"Unnamed: 0": "Group A"})
    df_splitB = df_splitB.rename(columns={"Unnamed: 0": "Group B"})
    df_splitA = pd.merge(df_splitA, df_split, on="Group A")
    df_splitB = pd.merge(df_splitB, df_split, on="Group B")
    df_splitA = df_splitA.drop(columns=["Group B"])
    df_splitB = df_splitB.drop(columns=["Group A"])
    
    #adding TP-FP to the group dataframes
    df_splitA =  add_TPFP_df(len(df_splitA.index), df_splitA)
    df_splitB =  add_TPFP_df(len(df_splitB.index), df_splitB)

    # Get the series top 10 of both split dataframes, get the top 10 data
    tserA = df_splitA.iloc[(len(df_splitA.index) - 1), 1:1411].sort_values(ascending = False)
    tserB = df_splitB.iloc[(len(df_splitB.index) - 1), 1:1411].sort_values(ascending = False)
    tdataA = [[tserA.index[0], tserA.iloc[0]], [tserA.index[1], tserA.iloc[1]], [tserA.index[2], tserA.iloc[2]], [tserA.index[3], tserA.iloc[3]], [tserA.index[4], tserA.iloc[4]], [tserA.index[5], tserA.iloc[5]],
        [tserA.index[6], tserA.iloc[6]], [tserA.index[7], tserA.iloc[7]], [tserA.index[8], tserA.iloc[8]], [tserA.index[9], tserA.iloc[9]]]
    tdataB = [[tserB.index[0], tserB.iloc[0]], [tserB.index[1], tserB.iloc[1]], [tserB.index[2], tserB.iloc[2]], [tserB.index[3], tserB.iloc[3]], [tserB.index[4], tserB.iloc[4]], [tserB.index[5], tserB.iloc[5]],[tserB.index[6], tserB.iloc[6]], [tserB.index[7], tserB.iloc[7]], [tserB.index[8], tserB.iloc[8]], [tserB.index[9], tserB.iloc[9]]]
    
    # Create top 10 dataframes 
    tempTTA=  pd.DataFrame(tdataA, columns = ['Mutation', 'TP-FP'])
    tempTTB = pd.DataFrame(tdataB, columns = ['Mutation', 'TP-FP'])
    
    root = Node(search_mutIndex(tempTTA.iloc[0,0], tempdf))
    root.left = Node(search_mutIndex(tempTTA.iloc[1,0], tempdf))
    root.right = Node(search_mutIndex(tempTTB.iloc[0,0], tempdf))
    root.left.left = Node(1)
    root.left.right = Node(0)
    root.right.left = Node(1)
    root.right.right = Node(0)
    
    return root

def make_treePhi(row_total, rootdf):
    # Create duplicate dataframes for root left and root right, keep track of their number of rows 
    root_L = rootdf
    root_R = rootdf
    split_rtotal = row_total
    
    # Check if the temp df has phi row, if it does drop the row in the group DFs and update the  split total 
    if(rootdf.iloc[((len(rootdf.index)) - 1), 0] == 'Phi'):
        root_L = root_L.drop(((len(rootdf.index)) - 1))
        root_R = root_R.drop(((len(rootdf.index)) - 1))
        split_rtotal -= 1 #54 delete later
    else: # This else statement has not been properly tested might be buggy!!!! Warning
        rootdf = add_phi_df(row_total, rootdf)
        row_total += 1
        
     # Get the top phi values in the rootdf, we will need this to get they key mutations
    index_last = row_total - 1
    split_groupDF = pd.DataFrame(columns = ['Root Left', 'Root Right'])
    tempSer = rootdf.iloc[index_last, 1:1411].sort_values(ascending = False)
    data = [[tempSer.index[0], tempSer.iloc[0]], [tempSer.index[1], tempSer.iloc[1]], [tempSer.index[2], tempSer.iloc[2]], [tempSer.index[3], tempSer.iloc[3]], [tempSer.index[4], tempSer.iloc[4]], [tempSer.index[5], tempSer.iloc[5]],
        [tempSer.index[6], tempSer.iloc[6]], [tempSer.index[7], tempSer.iloc[7]], [tempSer.index[8], tempSer.iloc[8]], [tempSer.index[9], tempSer.iloc[9]]]
    tempTTR = pd.DataFrame(data, columns = ['Mutation', 'Phi'])
    
    #Get the root phi name and feature index
    root_name = tempSer.index[0] 
    root_index = search_mutIndex(root_name, rootdf)
    
    
    # Partition the samples intto groups A and B 
    for i in range(index_last):
        if(rootdf.iloc[i, root_index] == 1.0):
            split_groupDF = split_groupDF.append({'Root Left':rootdf.iloc[i, 0]}, ignore_index = True)
        elif(rootdf.iloc[i, root_index] == 0.0):
            split_groupDF = split_groupDF.append({'Root Right':rootdf.iloc[i, 0]}, ignore_index = True)
        else:
            continue 
    # Setting up the dataframes for right and left root, renaming, merging and then dropping extra stuff           
    root_L = root_L.rename(columns={root_L.columns[0]: "Root Left"})
    root_R = root_R.rename(columns={root_R.columns[0]: "Root Right"})
    root_L = pd.merge(root_L, split_groupDF, on="Root Left")
    root_R = pd.merge(root_R, split_groupDF, on="Root Right")
    root_L = root_L.drop(columns=["Root Right"])
    root_R = root_R.drop(columns=["Root Left"]) 
    
    
    #adding phi to the group dataframes
    root_L =  add_phi_df(len(root_L.index), root_L)
    root_R =  add_phi_df(len(root_R.index), root_R)
    
    
    #Top 10 data of splits
    tserA = root_L.iloc[(len(root_L.index) - 1), 1:1411].sort_values(ascending = False)
    tserB = root_R.iloc[(len(root_R.index) - 1), 1:1411].sort_values(ascending = False)
    tdataA = [[tserA.index[0], tserA.iloc[0]], [tserA.index[1], tserA.iloc[1]], [tserA.index[2], tserA.iloc[2]], [tserA.index[3], tserA.iloc[3]], [tserA.index[4], tserA.iloc[4]], [tserA.index[5], tserA.iloc[5]],[tserA.index[6], tserA.iloc[6]], [tserA.index[7], tserA.iloc[7]], [tserA.index[8], tserA.iloc[8]], [tserA.index[9], tserA.iloc[9]]]
    tdataB = [[tserB.index[0], tserB.iloc[0]], [tserB.index[1], tserB.iloc[1]], [tserB.index[2], tserB.iloc[2]], [tserB.index[3], tserB.iloc[3]], [tserB.index[4], tserB.iloc[4]], [tserB.index[5], tserB.iloc[5]],[tserB.index[6], tserB.iloc[6]], [tserB.index[7], tserB.iloc[7]], [tserB.index[8], tserB.iloc[8]], [tserB.index[9], tserB.iloc[9]]]
   
    
    # Create top 10 dataframes 
    tempTTA=  pd.DataFrame(tdataA, columns = ['Mutation', 'Phi'])
    tempTTB = pd.DataFrame(tdataB, columns = ['Mutation', 'Phi']) 
    
    root = Node(search_mutIndex(tempTTR.iloc[0,0], rootdf))
    root.left = Node(search_mutIndex(tempTTA.iloc[0,0], rootdf))
    root.right = Node(search_mutIndex(tempTTB.iloc[0,0], rootdf))
    root.left.left = Node(1)
    root.left.right = Node(0)
    root.right.left = Node(1)
    root.right.right = Node(0)
    
    
    temp_list = [tempTTR, tempTTA, tempTTB, root_L, root_R, root]
    
    return temp_list
    
def num_samples(tempdf):
    return len((tempdf.index))

# The actual names of cancer samples in a dataframe
def act_totalC(tempdf):
    c_samp = 0
    for i in range (len(tempdf.index)): 
        if((tempdf.iloc[i,0][0] == 'C')):
            c_samp += 1 
    return c_samp

# The actual names of NC samples in a dataframe
def act_totalNC(tempdf):
    nc_samp = 0
    for i in range (len(tempdf.index)): 
        if((tempdf.iloc[i,0][0] == 'N')):
            nc_samp += 1 
    return nc_samp

# is we use a specific mutation to count C samples this is it
def numC_samples(tempdf, mutIndex):
    c_samp = 0
    if(mutIndex > 0):
        for i in range (len(tempdf.index)): 
            if((tempdf.iloc[i,mutIndex] == 1.0)):
                c_samp += 1 
    return c_samp

# if we use a specfic mutation to count NC samples
def numNC_samples(tempdf, mutIndex):
    nc_samp = 0
    if(mutIndex > 0):
        for i in range (len(tempdf.index)): 
            if((tempdf.iloc[i,mutIndex] == 0.0)):
                nc_samp += 1
    return nc_samp

def prob_Sample(child_df, rootdf):
    return (num_samples(child_df) / num_samples(rootdf))

def prob_selectC(tempdf):
    prob = 0
    div = num_samples(tempdf)
    if (div != 0):
        prob = act_totalC(tempdf) / num_samples(tempdf) 
    return prob 

def prob_selectNC(tempdf):
    prob = 0
    div = num_samples(tempdf)
    if (div != 0):
        prob = act_totalNC(tempdf) / num_samples(tempdf)
    return prob 

def compOne_phi(leftdf, rightdf, rootdf):
    val = 2 * prob_Sample(leftdf, rootdf) * prob_Sample(rightdf, rootdf)
    return val

def Qcomp_phi(leftdf, rightdf):
    val = (abs((prob_selectC(leftdf)) - (prob_selectC(rightdf))) + abs((prob_selectNC(leftdf)) - (prob_selectNC(rightdf))))
    return val

#calculates phi function for one mutation
def phi_functionMut(leftdf, rightdf, rootdf):
    phi = (compOne_phi(leftdf, rightdf, rootdf) * Qcomp_phi(leftdf, rightdf))
    return phi

#calculates phi function for all the mutations, must have a phi value column
def phi_function(rowNum, root_df):
    for i in range (1411):
        if(i > 0):
            root_L = root_df
            root_R = root_df
            split_df = pd.DataFrame(columns = ['Root Left', 'Root Right'])
            for j in range(len(root_df.index)):
                if(root_df.iloc[j, i] == 1.0):
                    split_df = split_df.append({'Root Left':root_df.iloc[j,0]}, ignore_index = True)
                elif(root_df.iloc[j, i] == 0.0):
                    split_df = split_df.append({'Root Right':root_df.iloc[j,0]}, ignore_index = True)
                else:
                    continue  
              
            root_L = root_L.rename(columns={root_L.columns[0]: "Root Left"})
            root_R = root_R.rename(columns={root_R.columns[0]: "Root Right"})
            root_L = pd.merge(root_L, split_df, on="Root Left")
            root_R = pd.merge(root_R, split_df, on="Root Right")
            root_L = root_L.drop(columns=["Root Right"])
            root_R = root_R.drop(columns=["Root Left"])  
            root_df.iloc[rowNum, i] = phi_functionMut(root_L, root_R, root_df)
        else:
            continue
    
def add_phi_df(row_total, rootdf):
    rootdf = rootdf.append({rootdf.columns[0]:'Phi'}, ignore_index = True)
    phi_function(row_total,rootdf)
    return rootdf

#confusion matrix for classifer, untested could be buggy
def cm_Model(row_total, temp_bt ,tempdf):
    temp_cm = pd.DataFrame(index = ['Predicted C', 'Predicted NC'], columns = ['Actual C', 'Actual NC'])
    temp_cm.iloc[0,0] = classifier_TP(row_total, tempdf, temp_bt)
    temp_cm.iloc[1,0] = classifier_FN(row_total, tempdf, temp_bt)
    temp_cm.iloc[0,1] = classifier_FP(row_total, tempdf, temp_bt)
    temp_cm.iloc[1,1] = classifier_TN(row_total, tempdf, temp_bt)
    return temp_cm

#confusion matrix for classifer, untested could be buggy
def cm_Mutation(row_total, mutIndex, tempdf):
    temp_cm = pd.DataFrame(index = ['Predicted C', 'Predicted NC'], columns = ['Actual C', 'Actual NC'])
    temp_cm.iloc[0,0] = true_positves(row_total, mutIndex, tempdf)
    temp_cm.iloc[1,0] = false_negatives(row_total, mutIndex, tempdf)
    temp_cm.iloc[0,1] = false_positives(row_total, mutIndex, tempdf)
    temp_cm.iloc[1,1] = true_negatives(row_total, mutIndex, tempdf)
    
    return temp_cm

df = df.append({'Unnamed: 0':'TP-FP'}, ignore_index = True)
TPtakeFP(162 ,df)
randomDF = randomDF.sample(n=162, replace = False)

groupI_df = randomDF.iloc[:54]
groupJ_df = randomDF.iloc[54:108]
groupK_df = randomDF.iloc[108:162]

pd.set_option('display.max_rows', 162)
groupIJ_df = groupI_df.append(groupJ_df, ignore_index=True)
groupJK_df = groupJ_df.append(groupK_df, ignore_index=True)
groupKI_df = groupK_df.append(groupI_df, ignore_index=True)

df = df.drop(index=162)
df
groupIJ_df
groupJK_df
groupKI_df

initTree = make_treePhi(162, df)
